In [2]:
%load_ext autoreload
%autoreload 2

In [62]:
import pandas as pd
from extract_information import extract_information
import numpy as np

## Evaluating from CSV

In [57]:
data = pd.read_csv('data/labeled_eligibility.csv')
data

,situation,age,income_from_work,non-work_income,marital_status,resources,disability_status,years_worked,lifetime_earnings,railroad_retirement_benefits,medicare_tax,ssdi_covered,sga_ability,adjustment_to_other_work,condition_lasting_over_12_months
0,"\nPeer Specialist: ""Good afternoon! How can I ...",40,2500.0,NaN,Married,300,unclear,NaN,NaN,NaN,NaN,NaN,unclear,NaN,NaN
1,"\nPeer Specialist: ""Hello! I’m glad you reache...",50,1500.0,NaN,Married,700,yes,NaN,NaN,NaN,NaN,NaN,no,NaN,yes
2,"\nPeer Specialist: ""Hi! It’s great to meet you...",45,1200.0,NaN,Living with partner,600,yes,NaN,NaN,NaN,NaN,NaN,no,NaN,yes
3,"\nPeer Specialist: ""Good afternoon! How can I ...",62,NaN,1500.0,Married,1000,unclear,40.0,NaN,NaN,NaN,NaN,unclear,NaN,NaN
4,"\nPeer Specialist: ""Hello! How can I assist yo...",55,1000.0,NaN,Widowed,800,yes,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN
5,I'm 48 years old and have been running my own ...,48,3000.0,NaN,Married,1200,unclear,NaN,NaN,NaN,NaN,NaN,unclear,NaN,NaN
6,I’m 58 years old and currently working part-ti...,58,2800.0,NaN,NaN,500,yes,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN
7,I’m a 67-year-old retired teacher. I spent ove...,67,NaN,2300.0,Widowed,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,I am a 35-year-old single parent with two chil...,35,3500.0,NaN,NaN,800,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"I am a 60-year-old individual, currently worki...",60,1200.0,NaN,NaN,1500,no,20.0,NaN,NaN,NaN,NaN,yes,NaN,NaN


In [58]:
with open('data/labeled_eligibility.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = [row for row in reader]

In [48]:
predicted_situations = []
for row in data:
    situation = row['situation']
    info = extract_information(situation)
    predicted_situations.append(info)

{'age': '40', 'income_from_work': '2500', 'non-work_income': 'empty', 'marital_status': 'Married', 'resources': '300', 'disability_status': 'No', 'years_worked': 'Several (exact number not provided)', 'lifetime_earnings': 'empty', 'railroad_retirement_benefits': 'No', 'medicare_tax': 'No', 'ssdi_covered': 'No', 'sga_ability': 'Yes', 'adjustment_to_other_work': 'Yes', 'condition_lasting_over_12_months': 'No'}
{'age': '50', 'income_from_work': '1500', 'non-work_income': '', 'marital_status': 'Married', 'resources': '700', 'disability_status': 'Yes', 'years_worked': '', 'lifetime_earnings': '', 'railroad_retirement_benefits': '', 'medicare_tax': '', 'ssdi_covered': '', 'sga_ability': '', 'adjustment_to_other_work': '', 'condition_lasting_over_12_months': 'Yes'}
{'age': '45', 'income_from_work': '1200', 'non-work_income': '', 'marital_status': 'Living with partner', 'resources': '600', 'disability_status': 'Yes', 'years_worked': '5', 'lifetime_earnings': '', 'railroad_retirement_benefits':

In [60]:
for i in predicted_situations:
    for j in i:
        if i[j] == 'empty':
            i[j] = ''

In [75]:
num_equal = 0
num_answered = np.zeros((2,2))
unclear_answered = 0

for i,d in enumerate(data):
    for key in d:
        if key != 'situation':
            if d[key] == predicted_situations[i][key]:
                num_equal += 1
            if d[key] == '' or d[key] == 'unclear':
                row = 0
            else:
                row = 1
            
            if predicted_situations[i][key] == '':
                column = 0
            else:
                column = 1

            if column == 1 and d[key] == 'unclear':
                unclear_answered += 1

            num_answered[row,column] += 1

In [76]:
# How often do LLM, ground truth agree?
num_equal/np.sum(num_answered)

0.5214285714285715

In [80]:
# When both ground-truth, LLM answer, how accurate is it? 
num_equal/(num_answered[0,0]+num_answered[1,1])

0.8588235294117647

In [77]:
# How often does LLM answer when ground truth isn't sure? 
num_answered[0,1]/np.sum(num_answered)

0.38571428571428573

In [78]:
# How often is the answer unclear? 
unclear_answered/np.sum(num_answered)

0.04285714285714286